ПОДГОТАВЛИВАЕМ ДАННЫЕ

In [1]:
from collections import Counter
import pandas as pd
import nltk
from nltk.corpus import stopwords
from sklearn.utils import shuffle

nltk.download('stopwords')

stop = set(stopwords.words("russian"))

def remove_stop_words(text):
    text = [word.lower() for word in text.split() if word.lower() not in stop]
    return " ".join(text)

messages = pd.read_csv("messages.csv", encoding="utf-8")
messages = messages[["id", "text"]]
messages.head()

labels = pd.read_csv("labels.csv", encoding="utf-8")
labels = labels[["id","label"]]
# labels['label'] = labels['label'].map({'other': 0, 'task': 1, 'event':2})
# print(labels.head())
messages["label"] = labels["label"].values

df_task = messages[messages["label"]=="task"]
df_task.shape

df_event = messages[messages["label"]=="event"]
df_event.shape

df_other = messages[messages["label"]=="other"]
df_other.shape

df_event_sample = df_event.sample(df_task.shape[0])
df_event_sample.shape

df_other_sample = df_other.sample(df_task.shape[0])
df_other_sample.shape

df_balanced = pd.concat([df_event_sample, df_other_sample,df_task])
df_balanced.shape
df_balanced["label"].value_counts()

messages = df_balanced


messages["text"] = messages["text"].map(remove_stop_words)
labels["label"] = "__label__" + labels["label"].astype(str)
messages["message_label"] = labels["label"] + " " + messages["text"]
messages = shuffle(messages)


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/taranyuriy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


СОЗДАЁМ МОДЕЛЬ

In [39]:
from sklearn.model_selection import train_test_split

train, test =  train_test_split(messages, test_size=0.3)

ЗАЧЕМ-ТО СОХРАНЯЕМ В ФАЙЛ CSV

In [46]:
train.to_csv("messages.train",columns=["message_label"],index=False, header=False)
test.to_csv("messages.test",columns=["message_label"],index=False, header=False)


In [1]:
import fasttext
import fasttext.util
# from sklearn.metrics import roc_auc_score

fasttext.util.download_model('ru', if_exists='ignore')
model = fasttext.train_supervised(input="messages.train",epoch=10, lr=1.0)
model.test("messages.test")
# model.predict("отправь мне почту завтра в 19:00")

AttributeError: module 'fasttext' has no attribute 'util'

In [52]:
model.predict("Пришли мне 17.01 твою выпускную работу")

(('__label__other',), array([0.99999869]))